The standard normal approach assumes the underlying distribution to be standard normal, and corresponding to this assumption, spits an interval.  
In case of standard normal, it is a sort of known mean and known variance.  
For modelling purposes, the underlying data used is uniform(-pi/3, +pi/3).  


Behaviour of standard normal distribution and student-t distribution:  
Considering 95% confidence => alpha = 0.975   
standard normal for all n : 1.96  

n = 10 => t : 2.228  
n = 15 => t : 2.131  
n = 20 => t: 2.086  
n = 50 => t : 2.009  
n = 100 => t : 1.984  
As n increases, the value from student t table approaches standard normal table.  
Considering input sample size to be of 15 (n=15); so that variability in confidence intervals of different approaches can be easily observed. 

In [1]:
import random
import statistics
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from scipy.stats import norm, t
import math
import mplcursors

def bootstrap(zdata,nreal):
    realisations = []
    zreal = []                            # declare an empty list to store the bootstrap repliations which contain mean of the 
                                          # bootstraped samples.
    for l in range(nreal):              # loop over the L bootstrap realizations
        samples = random.choices(zdata, k=len(zdata)) # n Monte Carlo simulations, sample with replacement
        realisations.append(samples)
        zreal.append(statistics.mean(samples))       # calculate the realization of the statistic and append to list
    return [samples, zreal]                          # return the list of realizations of the statistic

def get_95_percentile_points(distribution='normal', df=None):
    alpha_values = [0.025, 0.975]  # For 95% confidence interval
    if distribution == 'normal':
        z_values = [norm.ppf(alpha) for alpha in alpha_values]
    elif distribution == 't' and df is not None:
        z_values = [t.ppf(alpha, df) for alpha in alpha_values]
    else:
        raise ValueError("Invalid distribution type or missing degrees of freedom for t-distribution.")
    return z_values

In [2]:
# Reading the input file

observedSamples = []
with open('delta_theta_uniform_initial.txt', 'r') as file:
    for line in file:
        num_strings = line.strip().split(", ")
        nums = [round(float(num), 5) for num in num_strings]
        observedSamples.extend(nums)
BootstrapedSamples, MeanOfBootstrapedSamples = bootstrap(observedSamples, 1000) #No. of bootsraped samples is 1000.

observedSamples_df_1000 = []
with open('df_is_1000.txt', 'r') as file:
    for line in file:
        num_strings = line.strip().split(", ")
        nums = [round(float(num), 5) for num in num_strings]
        observedSamples_df_1000.extend(nums)
print(observedSamples_df_1000)
BootstrapedSamples_for_1000_df, MeanOfBootstrapedSamples_for_1000_df = bootstrap(observedSamples_df_1000, 1000) #No. of bootsraped samples is 1000.



[-0.78955, 0.19791, -0.74579, -0.18308, -0.20509, -0.46446, -0.2372, -0.69018, -0.61964, -0.5206, -0.05388, -0.25221, 0.83341, -0.50465, -0.21878, 0.36718, 0.02098, -0.18524, -0.83748, -0.56284, -0.72398, -0.66846, -0.03114, 0.47464, 0.32176, 0.39194, 0.38065, 0.73446, 0.16616, -0.25699, -0.63649, -0.94266, -0.50672, -0.09472, -0.97663, 0.15956, -1.03581, -0.36229, 0.52945, 0.68411, 0.88278, -0.53309, 0.63983, -0.02253, 0.9727, -0.56364, 0.66577, 0.15941, 0.18364, -0.97163, -0.25659, 0.61026, -0.84101, 0.01466, -0.39806, -0.52586, 0.95905, 0.22652, -0.57838, 1.01571, -0.0641, 0.18988, 0.41737, 0.52003, -0.14285, 0.07316, 0.3011, 0.86493, 0.9577, -0.16432, -0.51033, -0.08211, 0.68952, -0.90543, 0.02379, 0.38152, 0.61872, -0.54173, 0.3768, -0.1397, -0.934, 0.60276, -0.2631, 0.03273, 0.88508, 0.65435, -0.13985, 0.69145, -0.71453, -0.85187, 0.7244, -0.14693, -0.65899, -0.65678, -0.95873, -1.02139, -0.51963, 0.88616, -0.04143, -0.80938, -0.94748, 0.89121, -1.00031, 0.60425, -0.82903, -0.786

Calculating the standard error.

Method 1. Approch to find standard error as suggested by book.

In [3]:
valToBeSubtracted = sum(MeanOfBootstrapedSamples)/len(MeanOfBootstrapedSamples)
standardError = math.sqrt(sum((item - valToBeSubtracted) ** 2 for item in MeanOfBootstrapedSamples) / (len(MeanOfBootstrapedSamples) - 1))
print(standardError)

0.17792400785089055


Method 2. In the context of bootstrap resampling, the standard deviation of the bootstrap sample means directly represents the standard error of the original sample mean.

In [4]:
standard_error = statistics.stdev(MeanOfBootstrapedSamples)
print(standard_error)

0.17792400785089058


The confidence interval can be written as:
1. In case of standard normal.

$$
\hat{\theta} \pm Z^{(1-\alpha)} \cdot \hat{\text{se}}_B\\

\implies \theta \in \left[ \hat{\theta} - Z^{(1-\alpha)} \cdot \hat{\text{se}}_B, \; \hat{\theta} - Z^{\alpha} \cdot \hat{\text{se}}_B \right]
$$
where, $\newline$
&nbsp;&nbsp;&nbsp;&nbsp;$Z^{(1-\alpha)} = -Z^{\alpha}$ $\newline$
&nbsp;&nbsp;&nbsp;&nbsp;$Z^{\alpha}$ represents the 100.$\alpha^{th}$ percentile point of N(0, 1) distribution. $\newline$
&nbsp;&nbsp;&nbsp;&nbsp;$\hat{\theta}$ represents the plug-in estimate (mean of the given sample).$\newline$
&nbsp;&nbsp;&nbsp;&nbsp;$\theta$ represents the true mean of the unkown underlying distribution.$\newline$
&nbsp;&nbsp;&nbsp;&nbsp;$\hat{\text{se}}_B$ is the standard error calculated above.

2. In case of student-t distribution.

$$
\hat{\theta} \pm t_{n-1}^{(1-\alpha)} \cdot \hat{\text{se}}_B\\

\implies \theta \in \left[ \hat{\theta} - t_{n-1}^{(1-\alpha)} \cdot \hat{\text{se}}_B, \; \hat{\theta} - t_{n-1}^{\alpha} \cdot \hat{\text{se}}_B \right]
$$
where, $\newline$
&nbsp;&nbsp;&nbsp;&nbsp;n is the degrees of freedom (given sample's size) $\newline$
&nbsp;&nbsp;&nbsp;&nbsp;$t_{n-1}^{(1-\alpha)} = -t_{n-1}^{\alpha}$ $\newline$
&nbsp;&nbsp;&nbsp;&nbsp;$t^{\alpha}$ represents the 100.$\alpha^{th}$ percentile point of student-t distribution. $\newline$
&nbsp;&nbsp;&nbsp;&nbsp;$\hat{\theta}$ represents the plug-in estimate (mean of the given sample).$\newline$
&nbsp;&nbsp;&nbsp;&nbsp;$\theta$ represents the true mean of the unknown underlying distribution.$\newline$
&nbsp;&nbsp;&nbsp;&nbsp;$\hat{\text{se}}_B$ is the standard error calculated above.

More about t-distribution:  
1. Mean = 0  
2. Degrees of Freedom (df or $\nu$) = number of independent values or quantities that can vary in a statistical calculation.  
&nbsp;&nbsp;[In our case, df is same as the sample size, as all the values in the sample will change after bootstraping.]$\newline$
3.
$$ Variance =
\begin{cases} 
\frac{\nu}{\nu - 2} & \text{for } \nu > 2 \\
\infty & \text{for } \nu < 2
\end{cases}$$

4. For large $\nu$, variance = 1; resulting in $\mathcal{N}(0, 1)$.  
5. n=1 forms Standard Cauchy Distribution.



In [5]:
# Mean as the plug in estimate
plugInEstimate = statistics.mean(observedSamples)
# plugInEstimate_df_1000 = statistics.mean(observedSamples_df_1000)
# If we want a raw comparison between all, then we will assume that  there will exist a given data of size 1000, which has the 
# same mean as the mean of the given data of 15 entries. This is done so that the mean position about which the confidence 
# interval is made remains same; for an ideal visual comparison to be made.
plugInEstimate_df_1000 = plugInEstimate   


# For standard normal distribution
normal_z_values = get_95_percentile_points('normal')

print("Standard Normal Distribution Percentile Points for 95% confidence interval:")
for alpha, z in zip([0.025, 0.975], normal_z_values):
    print(f"z({alpha}) = {z:.3f}")

# For Student's t-distribution
df = [15, 1000]
# Case 1 : 15 degrees of freedom i.e. no. of observed values is 15.
t_z_values_15 = get_95_percentile_points('t', df[0])
print("\nStudent's t-Distribution (df=15) Percentile Points for 95% confidence interval (df=10):")
for alpha, z in zip([0.025, 0.975], t_z_values_15):
    print(f"t({alpha}) = {z:.3f}")

# Case 2 : 1000 degrees of freedom i.e. no. of observed values is 1000.
# Just to compare how student's t - prediction move towards normal distribution as no. of observed values increase.
t_z_values_1000 = get_95_percentile_points('t', df[1])

print("\nStudent's t-Distribution (df=1000) Percentile Points for 95% confidence interval (df=10):")
for alpha, z in zip([0.025, 0.975], t_z_values_1000):
    print(f"t({alpha}) = {z:.3f}")

# Calculate confidence intervals
normal_confidence_interval = [
    plugInEstimate - normal_z_values[0] * standardError,
    plugInEstimate - normal_z_values[1] * standardError
]

t_confidence_interval_15 = [
    plugInEstimate - t_z_values_15[0] * standardError,
    plugInEstimate - t_z_values_15[1] * standardError
]

t_confidence_interval_1000 = [
    plugInEstimate_df_1000 - t_z_values_1000[0] * standardError,
    plugInEstimate_df_1000 - t_z_values_1000[1] * standardError
]

print("\n95% Confidence Interval (Standard Normal Distribution):", normal_confidence_interval)
print("95% Confidence Interval (Student's t-Distribution, df = 15):", t_confidence_interval_15)
print("95% Confidence Interval (Student's t-Distribution, df = 1000):", t_confidence_interval_1000)



Standard Normal Distribution Percentile Points for 95% confidence interval:
z(0.025) = -1.960
z(0.975) = 1.960

Student's t-Distribution (df=15) Percentile Points for 95% confidence interval (df=10):
t(0.025) = -2.131
t(0.975) = 2.131

Student's t-Distribution (df=1000) Percentile Points for 95% confidence interval (df=10):
t(0.025) = -1.962
t(0.975) = 1.962

95% Confidence Interval (Standard Normal Distribution): [np.float64(0.3948179807061007), np.float64(-0.302631314039434)]
95% Confidence Interval (Student's t-Distribution, df = 15): [np.float64(0.4253293790112074), np.float64(-0.33314271234454074)]
95% Confidence Interval (Student's t-Distribution, df = 1000): [np.float64(0.39524056735640045), np.float64(-0.30305390068973376)]


One can observe that confidence interval in case of t-distrib. with a large degrees of freedom is equivalent to standard normal.  
  
why???

1. Central Limit Theorem:

According to the Central Limit Theorem, as the sample size increases, the sampling distribution of the sample mean approaches a normal distribution, regardless of the shape of the population distribution.

2. t-Distribution:

The t-distribution is used when estimating the population mean from a small sample size, and it accounts for the additional variability due to the small sample size. 

As the degrees of freedom (df) increase, the t-distribution approaches the standard normal distribution. This is because the additional variability (captured by the heavier tails) becomes less significant as the sample size increases, making the t-distribution narrower and more similar to the standard normal distribution.

Plotting

In [6]:
# Plotting with Plotly
# Standard Normal Distribution plot
x_norm = np.linspace(-4, 4, 1000)
y_norm = norm.pdf(x_norm)
fig = go.Figure()

fig.add_trace(go.Scatter(x=x_norm, y=y_norm, mode='lines', name='Standard Normal Distribution'))
fig.add_trace(go.Scatter(x=[normal_confidence_interval[0], normal_confidence_interval[0]], y=[0, norm.pdf(normal_confidence_interval[0])], mode='lines', name='95% CI Lower Bound', line=dict(color='red', dash='dash')))
fig.add_trace(go.Scatter(x=[normal_confidence_interval[1], normal_confidence_interval[1]], y=[0, norm.pdf(normal_confidence_interval[1])], mode='lines', name='95% CI Upper Bound', line=dict(color='red', dash='dash')))
fig.add_trace(go.Scatter(x=[normal_confidence_interval[0], normal_confidence_interval[1]], y=[0, 0], mode='lines', fill='tonexty', fillcolor='rgba(255,0,0,0.1)', showlegend=False))

# Add permanent labels for the confidence interval points with values
fig.add_trace(go.Scatter(
    x=[normal_confidence_interval[0]],
    y=[0],  # Position the label slightly below the top of the curve
    mode='markers+text',
    text=[f'[{normal_confidence_interval[0]:.2f}]'],
    textposition='top left',
    marker=dict(color='red', size=3),
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=[normal_confidence_interval[1]],
    y=[0.0001],  # Position the label slightly below the top of the curve
    mode='markers+text',
    text=[f'[{normal_confidence_interval[1]:.2f}]'],
    textposition='top right',
    marker=dict(color='red', size=3),
    showlegend=False
))

fig.update_layout(title='Standard Normal Distribution with 95% Confidence Interval', xaxis_title='x', yaxis_title='Probability Density')

# Save and open the figure in a web browser
import plotly.io as pio
pio.write_html(fig, 'standard_normal_distribution.html')


# Student's t-Distribution (df = 15) plot
x_t = np.linspace(-4, 4, 1000)
y_t = t.pdf(x_t, df[0])
fig2 = go.Figure()

fig2.add_trace(go.Scatter(x=x_t, y=y_t, mode='lines', name=f"Student's t-Distribution (df={df[0]})"))
fig2.add_trace(go.Scatter(x=[t_confidence_interval_15[0], t_confidence_interval_15[0]], y=[0, t.pdf(t_confidence_interval_15[0], df[0])], mode='lines', name='95% CI Lower Bound', line=dict(color='red', dash='dash')))
fig2.add_trace(go.Scatter(x=[t_confidence_interval_15[1], t_confidence_interval_15[1]], y=[0, t.pdf(t_confidence_interval_15[1], df[0])], mode='lines', name='95% CI Upper Bound', line=dict(color='red', dash='dash')))
fig2.add_trace(go.Scatter(x=[t_confidence_interval_15[0], t_confidence_interval_15[1]], y=[0, 0], mode='lines', fill='tonexty', fillcolor='rgba(255,0,0,0.1)', showlegend=False))

# Add permanent labels for the confidence interval points with values
fig2.add_trace(go.Scatter(
    x=[t_confidence_interval_15[0]],
    y=[0],  # Position the label slightly below the top of the curve
    mode='markers+text',
    text=[f'[{t_confidence_interval_15[0]:.2f}]'],
    textposition='top left',
    marker=dict(color='red', size=3),
    showlegend=False
))

fig2.add_trace(go.Scatter(
    x=[t_confidence_interval_15[1]],
    y=[0.0001],  # Position the label slightly below the top of the curve
    mode='markers+text',
    text=[f'[{t_confidence_interval_15[1]:.2f}]'],
    textposition='top right',
    marker=dict(color='red', size=3),
    showlegend=False
))

fig2.update_layout(title=f"Student's t-Distribution with 95% Confidence Interval (df={df[0]})", xaxis_title='x', yaxis_title='Probability Density')

# Save and open the figure in a web browser
import plotly.io as pio
pio.write_html(fig2, 'student_t_distribution_15.html')

# Student's t-Distribution (df = 1000) plot
x_t = np.linspace(-4, 4, 1000)
y_t = t.pdf(x_t, df[1])
fig3 = go.Figure()

fig3.add_trace(go.Scatter(x=x_t, y=y_t, mode='lines', name=f"Student's t-Distribution (df={df[1]})"))
fig3.add_trace(go.Scatter(x=[t_confidence_interval_1000[0], t_confidence_interval_1000[0]], y=[0, t.pdf(t_confidence_interval_1000[0], df[1])], mode='lines', name='95% Confidence Interval Lower Bound', line=dict(color='red', dash='dash')))
fig3.add_trace(go.Scatter(x=[t_confidence_interval_1000[1], t_confidence_interval_1000[1]], y=[0, t.pdf(t_confidence_interval_1000[1], df[1])], mode='lines', name='95% Confidence Interval Upper Bound', line=dict(color='red', dash='dash')))
fig3.add_trace(go.Scatter(x=[t_confidence_interval_1000[0], t_confidence_interval_1000[1]], y=[0, 0], mode='lines', fill='tonexty', fillcolor='rgba(255,0,0,0.1)', showlegend=False))

# Add permanent labels for the confidence interval points with values
fig3.add_trace(go.Scatter(
    x=[t_confidence_interval_1000[0]],
    y=[0],  # Position the label slightly below the top of the curve
    mode='markers+text',
    text=[f'[{t_confidence_interval_1000[0]:.2f}]'],
    textposition='top left',
    marker=dict(color='red', size=3),
    showlegend=False
))

fig3.add_trace(go.Scatter(
    x=[t_confidence_interval_1000[1]],
    y=[0.0001],  # Position the label slightly below the top of the curve
    mode='markers+text',
    text=[f'[{t_confidence_interval_1000[1]:.2f}]'],
    textposition='top right',
    marker=dict(color='red', size=3),
    showlegend=False
))

fig3.update_layout(title=f"Student's t-Distribution with 95% Confidence Interval (df={df[1]})", xaxis_title='x', yaxis_title='Probability Density')

# Save and open the figure in a web browser
pio.write_html(fig3, 'student_t_distribution_1000.html')

Conclusion:  
student's t provide a bigger interval than std. normal for df = 15.  
but as df increases, the student't interval tends to std. normal interval.  
df => Degrees of Freedom